In [14]:
def calc_principal_stresses(s):
    eigenValues, eigenVectors = np.linalg.eig(s)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return eigenValues, eigenVectors

In [1]:
def plot_interactive_initial():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description=r'$c$ / MPa'),
              phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$')
    )
    def plot_MC_mob(phi,beta,c):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)
        s_n = np.linspace(-0.3/np.tan(phi),2)
        
        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        

        fig, ax = plt.subplots(figsize=(18,10))
        #Mobilisierter Reibungswinkel
        tau_mob = lambda sn: np.maximum(sn*np.tan(beta),0)
        ax.plot(s_n,tau_mob(s_n),label=r'$\tau^\beta = \sigma_\mathrm{n}^\beta \mathrm{tan} \beta$',color='green')
        ax.plot(s_n,-tau_mob(s_n),color='green')
        ax.plot(1,tau_beta,marker='o',ls='',color='green',label=r'$(\sigma_\mathrm{n}^\beta,\tau^\beta$)')
        
        #Grenzzustand
        tau_f = lambda sn: np.maximum(c + sn*np.tan(phi),0)
        ax.plot(s_n,tau_f(s_n),label=r'$\tau^\mathrm{f} = c + \sigma_\mathrm{n} \mathrm{tan} \varphi$',color='red')
        ax.plot(s_n,-tau_f(s_n),color='red')
        
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel(r'$\sigma_\mathrm{n}$ / $\gamma z$')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel(r'$\tau$ / $\gamma z$')
        fig.legend()
        fig.tight_layout()
        plt.show();

In [ ]:
def identify_circle(p, *data):
    beta, phi, c = data
    sm, tmax = p
    return ((1 - sm)**2 + np.tan(beta)**2 - tmax**2, 
            (sm - tmax*np.sin(phi) - sm)**2 + (c + (sm - tmax*np.sin(phi))*np.tan(phi))**2 - tmax**2)

In [ ]:
def plot_interactive_circle():
    @interact(phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$'),
              show_active = widgets.Checkbox(value=True, description=r'aktiv'),
              show_passive = widgets.Checkbox(value=False, description=r'passiv'),
    )
    def plot_MC_mob(phi,beta,c,show_active,show_passive):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)
        fig, ax = plt.subplots(figsize=(18,10))

        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        
        #aktiver Spannungskreis
        if show_active:
            sma, tmaxa = fsolve(identify_circle, (0.5, 0.5), args=(beta,phi,c))
            MCa = patches.Circle((sma, 0.), radius=tmaxa, angle=90.,fill=False,lw=2)
            ax.add_patch(MCa)
        
        #passiver Spannungskreis
        if show_passive:
            smp, tmaxp = fsolve(identify_circle, (2, 2), args=(beta,phi,c))
            MCp = patches.Circle((smp, 0.), radius=tmaxp, angle=90.,fill=False,lw=2)
            ax.add_patch(MCp)
        
        s_n = np.linspace(-0.3/np.tan(phi),2)
        if show_passive:
            s_n2 = np.linspace(-0.3/np.tan(phi),smp+tmaxp)
        else:
            s_n2 = s_n
        
        #Mobilisierter Reibungswinkel
        tau_mob = lambda sn: np.maximum(sn*np.tan(beta),0)
        ax.plot(s_n2,tau_mob(s_n2),label=r'$\tau^\beta = \sigma_\mathrm{n}^\beta \mathrm{tan} \beta$',color='green')
        ax.plot(s_n2,-tau_mob(s_n2),color='green')
        ax.plot(1,tau_beta,marker='o',ls='',color='green',label=r'$(\sigma_\mathrm{n}^\beta,\tau^\beta$)')
        
        #Grenzzustand
        tau_f = lambda sn: np.maximum(c + sn*np.tan(phi),0)
        ax.plot(s_n,tau_f(s_n),label=r'$\tau^\mathrm{f} = c + \sigma_\mathrm{n} \mathrm{tan} \varphi$',color='red')
        ax.plot(s_n,-tau_f(s_n),color='red')
        
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel(r'$\sigma_\mathrm{n}$ / $\gamma z$')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel(r'$\tau$ / $\gamma z$')
        fig.legend()
        fig.tight_layout()
        plt.show();

In [2]:
def plot_interactive_slip():
    @interact(phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$'),
              show_active = widgets.Checkbox(value=True, description=r'aktiv'),
              show_passive = widgets.Checkbox(value=True, description=r'passiv'),
    )
    def plot_MC_mob(phi,beta,c,show_active,show_passive):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)
        fig, ax = plt.subplots(figsize=(18,10))

        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        
        #aktiver Spannungskreis
        if show_active:
            sma, tmaxa = fsolve(identify_circle, (0.5, 0.5), args=(beta,phi,c))
            MCa = patches.Circle((sma, 0.), radius=tmaxa, angle=90.,fill=False,lw=2)
            ax.add_patch(MCa)
            
            #Grenzpunkte
            sfa = sma - tmaxa * np.sin(phi)
            tfa = tmaxa * np.cos(phi)
            ax.plot(sfa,tfa,marker='o',ls='',color='blue',label=r'$(\sigma_\mathrm{f,a},\tau_\mathrm{f,a}$)')
            ax.plot(sfa,-tfa,marker='o',ls='',color='blue')
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((sma**2 - (1+np.tan(beta)**2)*(sma**2 - tmaxa**2))/(1+np.tan(beta)**2)**2)
            spa = sma/(1+np.tan(beta)**2) - wurzel
            ax.plot(spa,spa*np.tan(beta),marker='o',ls='',color='red',label=r'$P_\mathrm{a}$')
            
            #Gleitebenen
            ax.plot([spa,sfa],[spa*np.tan(beta),tfa],ls='--',color='blue',label=r'Gleitebene, aktiv')
            ax.plot([spa,sfa],[spa*np.tan(beta),-tfa],ls='--',color='blue')
        
        #passiver Spannungskreis
        if show_passive:
            smp, tmaxp = fsolve(identify_circle, (2, 2), args=(beta,phi,c))
            MCp = patches.Circle((smp, 0.), radius=tmaxp, angle=90.,fill=False,lw=2)
            ax.add_patch(MCp)
            
            #Grenzpunkte
            sfp = smp - tmaxp * np.sin(phi)
            tfp = tmaxp * np.cos(phi)
            ax.plot(sfp,tfp,marker='d',ls='',color='blue',label=r'$(\sigma_\mathrm{f,p},\tau_\mathrm{f,p}$)')
            ax.plot(sfp,-tfp,marker='d',ls='',color='blue')
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((smp**2 - (1+np.tan(beta)**2)*(smp**2 - tmaxp**2))/(1+np.tan(beta)**2)**2)
            spp = smp/(1+np.tan(beta)**2) + wurzel
            ax.plot(spp,spp*np.tan(beta),marker='d',ls='',color='red',label=r'$P_\mathrm{p}$')
            
            #Gleitebenen
            ax.plot([spp,sfp],[spp*np.tan(beta),tfp],ls=':',color='blue',label=r'Gleitebene, passiv')
            ax.plot([spp,sfp],[spp*np.tan(beta),-tfp],ls=':',color='blue')
        
        s_n = np.linspace(-0.3/np.tan(phi),2)
        if show_passive:
            s_n2 = np.linspace(-0.3/np.tan(phi),smp+tmaxp)
        else:
            s_n2 = s_n
        
        #Mobilisierter Reibungswinkel
        tau_mob = lambda sn: np.maximum(sn*np.tan(beta),0)
        ax.plot(s_n2,tau_mob(s_n2),label=r'$\tau^\beta = \sigma_\mathrm{n}^\beta \mathrm{tan} \beta$',color='green')
        ax.plot(s_n2,-tau_mob(s_n2),color='green')
        ax.plot(1,tau_beta,marker='o',ls='',color='green',label=r'$(\sigma_\mathrm{n}^\beta,\tau^\beta$)')
        
        #Grenzzustand
        tau_f = lambda sn: np.maximum(c + sn*np.tan(phi),0)
        ax.plot(s_n,tau_f(s_n),label=r'$\tau^\mathrm{f} = c + \sigma_\mathrm{n} \mathrm{tan} \varphi$',color='red')
        ax.plot(s_n,-tau_f(s_n),color='red')
        
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel(r'$\sigma_\mathrm{n}$ / $\gamma z$')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel(r'$\tau$ / $\gamma z$')
        fig.legend()
        fig.tight_layout()
        plt.show()

In [3]:
def plot_interactive_principal():
    @interact(phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$'),
              show_active = widgets.Checkbox(value=True, description=r'aktiv'),
              show_passive = widgets.Checkbox(value=True, description=r'passiv'),
    )
    def plot_MC_mob(phi,beta,c,show_active,show_passive):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)
        fig, ax = plt.subplots(figsize=(18,10))

        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        
        #aktiver Spannungskreis
        if show_active:
            sma, tmaxa = fsolve(identify_circle, (0.5, 0.5), args=(beta,phi,c))
            MCa = patches.Circle((sma, 0.), radius=tmaxa, angle=90.,fill=False,lw=2)
            ax.add_patch(MCa)
            
            #Hauptspannungspunkte
            s1a = sma + tmaxa
            s3a = sma - tmaxa
            ax.plot(s1a,0,marker='o',ls='',color='blue',label=r'$\sigma_{1\mathrm{a}}$')
            ax.plot(s3a,0,marker='o',ls='',color='orange',label=r'$\sigma_{3\mathrm{a}}$')
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((sma**2 - (1+np.tan(beta)**2)*(sma**2 - tmaxa**2))/(1+np.tan(beta)**2)**2)
            spa = sma/(1+np.tan(beta)**2) - wurzel
            ax.plot(spa,spa*np.tan(beta),marker='o',ls='',color='red',label=r'$P_\mathrm{a}$')
            
            #Hauptspannungsebenen
            ax.plot([spa,s1a],[spa*np.tan(beta),0],ls='--',color='blue')
            ax.plot([spa,s3a],[spa*np.tan(beta),0],ls='--',color='orange')
        
        #passiver Spannungskreis
        if show_passive:
            smp, tmaxp = fsolve(identify_circle, (2, 2), args=(beta,phi,c))
            MCp = patches.Circle((smp, 0.), radius=tmaxp, angle=90.,fill=False,lw=2)
            ax.add_patch(MCp)
            
            #Hauptspannungspunkte
            s1p = smp + tmaxp
            s3p = smp - tmaxp
            ax.plot(s1p,0,marker='d',ls='',color='blue',label=r'$\sigma_{1\mathrm{p}}$')
            ax.plot(s3p,0,marker='d',ls='',color='orange',label=r'$\sigma_{3\mathrm{p}}$')
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((smp**2 - (1+np.tan(beta)**2)*(smp**2 - tmaxp**2))/(1+np.tan(beta)**2)**2)
            spp = smp/(1+np.tan(beta)**2) + wurzel
            ax.plot(spp,spp*np.tan(beta),marker='d',ls='',color='red',label=r'$P_\mathrm{p}$')
            
            #Hauptspannungsebenen
            ax.plot([spp,s1p],[spp*np.tan(beta),0],ls=':',color='blue')
            ax.plot([spp,s3p],[spp*np.tan(beta),0],ls=':',color='orange')
        
        s_n = np.linspace(-0.3/np.tan(phi),2)
        if show_passive:
            s_n2 = np.linspace(-0.3/np.tan(phi),smp+tmaxp)
        else:
            s_n2 = s_n
        
        #Mobilisierter Reibungswinkel
        tau_mob = lambda sn: np.maximum(sn*np.tan(beta),0)
        ax.plot(s_n2,tau_mob(s_n2),label=r'$\tau^\beta = \sigma_\mathrm{n}^\beta \mathrm{tan} \beta$',color='green')
        ax.plot(s_n2,-tau_mob(s_n2),color='green')
        ax.plot(1,tau_beta,marker='o',ls='',color='green',label=r'$(\sigma_\mathrm{n}^\beta,\tau^\beta$)')
        
        #Grenzzustand
        tau_f = lambda sn: np.maximum(c + sn*np.tan(phi),0)
        ax.plot(s_n,tau_f(s_n),label=r'$\tau^\mathrm{f} = c + \sigma_\mathrm{n} \mathrm{tan} \varphi$',color='red')
        ax.plot(s_n,-tau_f(s_n),color='red')
        
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel(r'$\sigma_\mathrm{n}$ / $\gamma z$')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel(r'$\tau$ / $\gamma z$')
        fig.legend()
        fig.tight_layout()
        plt.show()

In [4]:
def plot_interactive_wall():
    @interact(phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$'),
              show_active = widgets.Checkbox(value=True, description=r'aktiv'),
              show_passive = widgets.Checkbox(value=True, description=r'passiv'),
    )
    def plot_MC_mob(phi,beta,c,show_active,show_passive):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)
        fig, ax = plt.subplots(figsize=(18,10))

        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        
        #aktiver Spannungskreis
        if show_active:
            sma, tmaxa = fsolve(identify_circle, (0.5, 0.5), args=(beta,phi,c))
            MCa = patches.Circle((sma, 0.), radius=tmaxa, angle=90.,fill=False,lw=2)
            ax.add_patch(MCa)
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((sma**2 - (1+np.tan(beta)**2)*(sma**2 - tmaxa**2))/(1+np.tan(beta)**2)**2)
            spa = sma/(1+np.tan(beta)**2) - wurzel
            ax.plot(spa,spa*np.tan(beta),marker='o',ls='',color='red',label=r'$P_\mathrm{a}$')
            
            #Wandspannung
            ax.plot([spa,spa],[spa*np.tan(beta),-spa*np.tan(beta)],ls='--',color='blue')
            ax.plot(spa,-spa*np.tan(beta),ls='',marker='o',color='blue',label=r'$e_\mathrm{a}$')
        
        #passiver Spannungskreis
        if show_passive:
            smp, tmaxp = fsolve(identify_circle, (2, 2), args=(beta,phi,c))
            MCp = patches.Circle((smp, 0.), radius=tmaxp, angle=90.,fill=False,lw=2)
            ax.add_patch(MCp)
            
            #Pol
            with np.errstate(divide='ignore', invalid='ignore'):
                wurzel = np.sqrt((smp**2 - (1+np.tan(beta)**2)*(smp**2 - tmaxp**2))/(1+np.tan(beta)**2)**2)
            spp = smp/(1+np.tan(beta)**2) + wurzel
            ax.plot(spp,spp*np.tan(beta),marker='d',ls='',color='red',label=r'$P_\mathrm{p}$')
            
            #Wandspannung
            ax.plot([spp,spp],[spp*np.tan(beta),-spp*np.tan(beta)],ls=':',color='blue')
            ax.plot(spp,-spp*np.tan(beta),ls='',marker='d',color='blue',label=r'$e_\mathrm{p}$')
        
        s_n = np.linspace(-0.3/np.tan(phi),2)
        if show_passive:
            s_n2 = np.linspace(-0.3/np.tan(phi),smp+tmaxp)
        else:
            s_n2 = s_n
        
        #Mobilisierter Reibungswinkel
        tau_mob = lambda sn: np.maximum(sn*np.tan(beta),0)
        ax.plot(s_n2,tau_mob(s_n2),label=r'$\tau^\beta = \sigma_\mathrm{n}^\beta \mathrm{tan} \beta$',color='green')
        ax.plot(s_n2,-tau_mob(s_n2),color='green')
        ax.plot(1,tau_beta,marker='o',ls='',color='green',label=r'$(\sigma_\mathrm{n}^\beta,\tau^\beta$)')
        
        #Grenzzustand
        tau_f = lambda sn: np.maximum(c + sn*np.tan(phi),0)
        ax.plot(s_n,tau_f(s_n),label=r'$\tau^\mathrm{f} = c + \sigma_\mathrm{n} \mathrm{tan} \varphi$',color='red')
        ax.plot(s_n,-tau_f(s_n),color='red')
        
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel(r'$\sigma_\mathrm{n}$ / $\gamma z$')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel(r'$\tau$ / $\gamma z$')
        fig.legend()
        fig.tight_layout()
        plt.show()

In [1]:
def summary_interactive_dual():
    @interact(phi=widgets.FloatSlider(value=30.,min=15, max=40, step=1, description=r'$\varphi$ / °'),
              beta=widgets.FloatSlider(value=8.,min=0, max=15, step=1, description=r'$\beta$ / °'),
              c=widgets.FloatSlider(value=0.2, step = 0.05, min=0, max=0.3, description=r'$c$ / $\gamma z$')
    )
    def summary(phi,beta,c):
        phi = np.deg2rad(phi)
        beta = np.deg2rad(beta)

        #Spannung auf Böschungslinie
        s_beta = 1
        tau_beta = np.tan(beta)
        
        #aktiver Spannungskreis
        sma, tmaxa = fsolve(identify_circle, (0.5, 0.5), args=(beta,phi,c))
        
        sfa = sma - tmaxa * np.sin(phi)
        tfa = tmaxa * np.cos(phi)
        
        s1a = sma + tmaxa
        s3a = sma - tmaxa

        #Pol
        with np.errstate(divide='ignore', invalid='ignore'):
            wurzel = np.sqrt((sma**2 - (1+np.tan(beta)**2)*(sma**2 - tmaxa**2))/(1+np.tan(beta)**2)**2)
        spa = sma/(1+np.tan(beta)**2) - wurzel
        
        theta_princ_a = np.rad2deg(np.arctan(spa*np.tan(beta)/(spa - s3a)))
        theta_a = np.pi/4 - phi/2
        theta_p = np.pi/4 - phi/2

        #passiver Spannungskreis
        smp, tmaxp = fsolve(identify_circle, (2, 2), args=(beta,phi,c))
        
        sfp = smp - tmaxp * np.sin(phi)
        tfp = tmaxp * np.cos(phi)
        
        s1p = smp + tmaxp
        s3p = smp - tmaxp

        #Pol
        with np.errstate(divide='ignore', invalid='ignore'):
            wurzel = np.sqrt((smp**2 - (1+np.tan(beta)**2)*(smp**2 - tmaxp**2))/(1+np.tan(beta)**2)**2)
        spp = smp/(1+np.tan(beta)**2) + wurzel
        
        theta_princ_p = np.rad2deg(np.arctan(spp*np.tan(beta)/(spp-s3p)))
        
        fig, ax = plt.subplots(ncols=2,figsize=(20,8))
        act = patches.Ellipse((0,0), s1a, s3a,
                     angle=theta_princ_a, linewidth=2, fill=False, color = 'blue')
        ax[0].add_patch(act)
        ax[0].plot([-s1a*np.cos(np.deg2rad(theta_princ_a))/2,s1a*np.cos(np.deg2rad(theta_princ_a))/2],
                [-s1a*np.sin(np.deg2rad(theta_princ_a))/2,s1a*np.sin(np.deg2rad(theta_princ_a))/2],color='blue',ls='--',
               label=r'Hauptachsen, aktiv')
        ax[0].plot([-s3a*np.cos(np.deg2rad(theta_princ_a)+np.pi/2)/2,s3a*np.cos(np.deg2rad(theta_princ_a)+np.pi/2)/2],
                [-s3a*np.sin(np.deg2rad(theta_princ_a)+np.pi/2)/2,s3a*np.sin(np.deg2rad(theta_princ_a)+np.pi/2)/2],color='blue',ls='--')
        ax[0].plot([-s1p*np.cos(np.deg2rad(theta_princ_a)+theta_a)/2,s1p*np.cos(np.deg2rad(theta_princ_a)+theta_a)/2],
                [-s1p*np.sin(np.deg2rad(theta_princ_a)+theta_a)/2,s1p*np.sin(np.deg2rad(theta_princ_a)+theta_a)/2],color='blue',ls=':',
               label=r'Gleitebenen, aktiv')
        ax[0].plot([-s1p*np.cos(np.deg2rad(theta_princ_a)-theta_a)/2,s1p*np.cos(np.deg2rad(theta_princ_a)-theta_a)/2],
                [-s1p*np.sin(np.deg2rad(theta_princ_a)-theta_a)/2,s1p*np.sin(np.deg2rad(theta_princ_a)-theta_a)/2],color='blue',ls=':')
        
        pas = patches.Ellipse((0,0), s1p, s3p,
                     angle=theta_princ_p, linewidth=2, fill=False, color = 'red')
        ax[0].add_patch(pas)
        ax[0].plot([-s1p*np.cos(np.deg2rad(theta_princ_p))/2,s1p*np.cos(np.deg2rad(theta_princ_p))/2],
                [-s1p*np.sin(np.deg2rad(theta_princ_p))/2,s1p*np.sin(np.deg2rad(theta_princ_p))/2],color='red',ls='--',
               label=r'Hauptachsen, passiv')
        ax[0].plot([-s3p*np.cos(np.deg2rad(theta_princ_p)+np.pi/2)/2,s3p*np.cos(np.deg2rad(theta_princ_p)+np.pi/2)/2],
                [-s3p*np.sin(np.deg2rad(theta_princ_p)+np.pi/2)/2,s3p*np.sin(np.deg2rad(theta_princ_p)+np.pi/2)/2],color='red',ls='--')
        ax[0].plot([-s1p*np.cos(np.deg2rad(theta_princ_p)+theta_p)/2,s1p*np.cos(np.deg2rad(theta_princ_p)+theta_p)/2],
                [-s1p*np.sin(np.deg2rad(theta_princ_p)+theta_p)/2,s1p*np.sin(np.deg2rad(theta_princ_p)+theta_p)/2],color='red',ls=':',
               label=r'Gleitebenen, passiv')
        ax[0].plot([-s1p*np.cos(np.deg2rad(theta_princ_p)-theta_p)/2,s1p*np.cos(np.deg2rad(theta_princ_p)-theta_p)/2],
                [-s1p*np.sin(np.deg2rad(theta_princ_p)-theta_p)/2,s1p*np.sin(np.deg2rad(theta_princ_p)-theta_p)/2],color='red',ls=':')
        
        ax[0].plot([-s1p,s1p],[-np.tan(beta)*s1p,np.tan(beta)*s1p],ls='-.',label=r'Geländelinie',color='green')
        
        ax[0].set_aspect('equal')
        ax[0].set_xlim(-s1p*0.61,s1p*0.61)
        ax[0].set_ylim(-s1p*0.61,s1p*0.61)
        ax[0].axis('off')
        ax[0].legend(ncol=3)
        
        ax[1].text(1,10,r'Aktiv',fontweight='bold')
        ax[1].text(1,9.5,r'$e_\mathrm{ah} = %.3f\, \gamma z$' %spa)
        ax[1].text(1,9,r'$e_\mathrm{av} = %.3f\, \gamma z$' %(spa*np.tan(beta)))
        ax[1].text(1,8.5,r'$\sigma_1 = %.3f\, \gamma z$' %s1a)
        ax[1].text(1,8,r'$\sigma_3 = %.3f\, \gamma z$' %s3a)
        ax[1].text(1,7.5,r'$\sigma_\mathrm{f} = %.3f\, \gamma z$' %sfa)
        ax[1].text(1,7,r'$\tau_\mathrm{f} = %.3f\, \gamma z$' %tfa)
        
        ax[1].text(5,10,r'Passiv',fontweight='bold')
        ax[1].text(5,9.5,r'$e_\mathrm{ph} = %.3f\, \gamma z$' %spp)
        ax[1].text(5,9,r'$e_\mathrm{pv} = %.3f\, \gamma z$' %(spp*np.tan(beta)))
        ax[1].text(5,8.5,r'$\sigma_1 = %.3f\, \gamma z$' %s1p)
        ax[1].text(5,8,r'$\sigma_3 = %.3f\, \gamma z$' %s3p)
        ax[1].text(5,7.5,r'$\sigma_\mathrm{f} = %.3f\, \gamma z$' %sfp)
        ax[1].text(5,7,r'$\tau_\mathrm{f} = %.3f\, \gamma z$' %tfp)
        ax[1].set_ylim(0,10)
        ax[1].set_xlim(0,10)
        ax[1].axis('off')
        fig.tight_layout()
        plt.show();